This notebook demonstrates how to process Sentinel-2 satellite imagery to calculate the Normalized Burn Ratio (NBR) and the delta Normalized Burn Ratio (dNBR) using the Google Earth Engine (GEE) Python API. 

- The **Normalized Burn Ratio (NBR)** is a remote sensing index used to identify areas affected by fire. It is computed using the near-infrared (NIR) and shortwave-infrared (SWIR) bands, which are sensitive to vegetation and burned areas.

- The **delta Normalized Burn Ratio (dNBR)** is the difference between pre-fire and post-fire NBR values, providing an effective metric for assessing burn severity.


## Workflow
1. Define the region of interest using a GeoJSON polygon.
2. Filter Sentinel-2 imagery by date and cloud cover.
3. Compute the NBR and dNBR indices.
4. Visualize the results with `geemap`

## Prerequisites

To run this notebook, you will need to have a Google Earth Engine account ([Sign up here](https://earthengine.google.com/)).

You will also need to ensure you have the necessary tools and libraries installed in a clean virtual environment. Follow the steps below to create and set up a virtual environment:  

### Install the dependencies

#### Using `venv` (Python Standard Library) 
1. **Create a virtual environment**:  
Ensure the desired Python version (e.g., Python 3.12) is installed on your system. Then run:

```bash  
   python3 -m venv nbr-env 
```
This creates a directory named nbr-env that contains the virtual environment.

1. **Activate the virtual environment**

- On macOS/Linux:
```bash

source nbr-env/bin/activate
```
- On Windows (Command Prompt):
```bash
nbr-env\Scripts\activate
```

3. **Install dependencies from requirements.txt**
```bash
pip install -r requirements.txt
```

####  Using conda (Anaconda/Miniconda)
1. Create a new Conda environment:

```bash
conda create --name nbr-env python=3.12
```
2. Activate the environment:

```bash
conda activate nbr-env
```

3. Use the following command to install dependencies:

```bash
pip install -r requirements.txt
```


Now you are all set up, let's start ! 

## Imports 

In [1]:
import ee
import json
import geemap

In [2]:
# Authenticate
ee.Authenticate()
# Initialize the Earth Engine API
ee.Initialize()

## Define the Region of Interest

In [3]:
# Load GeoJSON and and convert to Earth Engine geometry
def geojson_to_ee(file_path):
    with open(file_path, "r") as f:
        geojson_data = json.load(f)
    roi = ee.Geometry.Polygon(geojson_data["features"][0]["geometry"]["coordinates"])
    return roi


# Change the Geojson to your own need:
geojson_path = "./Los_Angeles.geojson"
roi = geojson_to_ee(geojson_path)
print("ROI set successfully:", roi.getInfo())

ROI set successfully: {'type': 'Polygon', 'coordinates': [[[-118.6102185066212, 33.64551991176346], [-117.82452679002279, 33.64551991176346], [-117.82452679002279, 34.288898731364455], [-118.6102185066212, 34.288898731364455], [-118.6102185066212, 33.64551991176346]]]}


## Data processing

As of now the NBR and dNBR is computed using only Sentinel-2 imagery. Implementation with Landsat 8 will follow. 

In [4]:
# Sentinel-2 Surface Reflectance
collection = "COPERNICUS/S2_SR_HARMONIZED"


def mask_s2_clouds(image):
    """Masks clouds in a Sentinel-2 image using the QA band.

    Args:
        image (ee.Image): A Sentinel-2 image.

    Returns:
        ee.Image: A cloud-masked Sentinel-2 image.
    """
    qa = image.select("QA60")

    # Bits 10 and 11 are clouds and cirrus, respectively.
    cloud_bit_mask = 1 << 10
    cirrus_bit_mask = 1 << 11

    # Both flags should be set to zero, indicating clear conditions.
    mask = qa.bitwiseAnd(cloud_bit_mask).eq(0).And(qa.bitwiseAnd(cirrus_bit_mask).eq(0))

    return image.updateMask(mask).divide(10000)


# Define a function to calculate NBR
def calculate_nbr(image):
    nbr = image.normalizedDifference(["B8", "B12"]).rename("NBR")
    return image.addBands(nbr)


# Retrieve Sentinel-2 images for a specific time range and region
def get_sentinel2_nbr(roi, start_date, end_date):
    sentinel2 = (
        ee.ImageCollection(collection)
        .filterBounds(roi)
        .filterDate(
            start_date, end_date
        )  # Use the full date range for pre- and post-event
        .map(mask_s2_clouds)
        .map(calculate_nbr)
    )
    # Check if the collection has valid images
    count = sentinel2.size().getInfo()
    if count == 0:
        raise ValueError(
            f"No images found for the date range {start_date} to {end_date}"
        )

    return sentinel2.mean()

In [5]:
# Define the pre-event and post-event dates
pre_fire_start_date = "2024-12-01"
pre_fire_end_date = "2025-01-01"
post_fire_start_date = "2025-01-10"
post_fire_end_date = "2025-01-20"

pre_event_image = get_sentinel2_nbr(roi, pre_fire_start_date, pre_fire_end_date)
print(pre_event_image.getInfo())

post_event_image = get_sentinel2_nbr(roi, post_fire_start_date, post_fire_end_date)
print(post_event_image.getInfo())

{'type': 'Image', 'bands': [{'id': 'B1', 'data_type': {'type': 'PixelType', 'precision': 'float', 'min': 0, 'max': 6.553500175476074}, 'crs': 'EPSG:4326', 'crs_transform': [1, 0, 0, 0, 1, 0]}, {'id': 'B2', 'data_type': {'type': 'PixelType', 'precision': 'float', 'min': 0, 'max': 6.553500175476074}, 'crs': 'EPSG:4326', 'crs_transform': [1, 0, 0, 0, 1, 0]}, {'id': 'B3', 'data_type': {'type': 'PixelType', 'precision': 'float', 'min': 0, 'max': 6.553500175476074}, 'crs': 'EPSG:4326', 'crs_transform': [1, 0, 0, 0, 1, 0]}, {'id': 'B4', 'data_type': {'type': 'PixelType', 'precision': 'float', 'min': 0, 'max': 6.553500175476074}, 'crs': 'EPSG:4326', 'crs_transform': [1, 0, 0, 0, 1, 0]}, {'id': 'B5', 'data_type': {'type': 'PixelType', 'precision': 'float', 'min': 0, 'max': 6.553500175476074}, 'crs': 'EPSG:4326', 'crs_transform': [1, 0, 0, 0, 1, 0]}, {'id': 'B6', 'data_type': {'type': 'PixelType', 'precision': 'float', 'min': 0, 'max': 6.553500175476074}, 'crs': 'EPSG:4326', 'crs_transform': [1,

In [6]:
# Calculate dNBR for the given dates and ROI

# Compute dNBR
dnbr = (
    pre_event_image.select("NBR")
    .subtract(post_event_image.select("NBR"))
    .multiply(1000)
    .rename("dNBR")
)

In [7]:
# Print the dNBR image details
print(dnbr.getInfo())

{'type': 'Image', 'bands': [{'id': 'dNBR', 'data_type': {'type': 'PixelType', 'precision': 'float', 'min': -2000, 'max': 2000}, 'crs': 'EPSG:4326', 'crs_transform': [1, 0, 0, 0, 1, 0]}]}


## Mapping the results

In [10]:
Map = geemap.Map()
Map.centerObject(roi, 10)

boundary_style = {
    "color": "ffffff",  # White color
    "strokeWidth": 5,  # Stroke width of 5
}
Map.addLayer(roi, boundary_style, "Study Area")

# Define visualization parameters for NBR (display range, color palette)
nbr_vis = {"min": -1, "max": 1, "palette": ["blue", "white", "green"]}

# Add pre-event and post-event NBR images to the map
Map.addLayer(pre_event_image.select("NBR"), nbr_vis, "Pre-Fire NBR")
Map.addLayer(post_event_image.select("NBR"), nbr_vis, "Post-Fire NBR")

# Define the color palette for the dNBR classification based on new categories
color_map = [
    "#7a8737",  # Enhanced Regrowth (> -0.1)
    "#acbe4d",  # Unburned (-0.1 to +0.1)
    "#0ae042",  # Low Severity (+0.1 to +0.27)
    "#fff70b",  # Moderate Severity Low  (+0.27 to +0.44)
    "#ffaf38",  # Moderate Severity High (+0.44 to +0.66)
    "#ff641b",  # High Severity (> +0.66) - Red
]

# Define the thresholds for the burn severity classes (scaled by 1000)
thresholds = [-100, 100, 270, 440, 660]  # Multiplying by 1000

# Classify the dNBR image based on the thresholds
classified = (
    dnbr.lt(thresholds[0])
    .add(dnbr.gte(thresholds[0]).And(dnbr.lt(thresholds[1])).multiply(2))
    .add(dnbr.gte(thresholds[1]).And(dnbr.lt(thresholds[2])).multiply(3))
    .add(dnbr.gte(thresholds[2]).And(dnbr.lt(thresholds[3])).multiply(4))
    .add(dnbr.gte(thresholds[3]).And(dnbr.lt(thresholds[4])).multiply(5))
    .add(dnbr.gte(thresholds[4]).multiply(6))
    .toInt()
)
# Define the legend dictionary
legend_dict = {
    "Enhanced Regrowth": "#7a8737",
    "Unburned": "#acbe4d",
    "Low Severity": "#0ae042",
    "Moderate-low Severity": "#fff70b",
    "Moderate-high Severity": "#ffaf38",
    "High Severity": "#ff641b",
}

# Define visualization parameters for the classified image
classified_vis = {
    "min": 0,
    "max": 5,  # Total number of classes
    "palette": list(legend_dict.values()),
}

# Add the classified dNBR image to the map (without the original dNBR)
Map.addLayer(classified, classified_vis, "dNBR Classified")


# Add the legend to the map (this is the only legend)
Map.add_legend(
    title="dNBR Classes",
    keys=list(legend_dict.keys()),
    colors=list(legend_dict.values()),
)

# Show the map
Map


Map(center=[33.96742776505625, -118.21737264832217], controls=(WidgetControl(options=['position', 'transparent…